# Lab 2 - Xử lý ảnh số
**Thực hiện đúng theo yêu cầu bài tập gồm 4 câu:**

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage, fft
import math
import os
import random

## Câu 1: Menu xử lý điểm ảnh (I, G, L, H, C)

In [ ]:

def image_inverse(img): return 255 - img
def gamma_correction(img, gamma=0.5):
    b = img.astype(float)
    b /= b.max()
    return (np.exp(np.log(b) * gamma) * 255).astype(np.uint8)
def log_transform(img):
    b = img.astype(float)
    return ((128 * np.log(1 + b)) / np.log(1 + b.max())).astype(np.uint8)
def histogram_equalization(img):
    hist, _ = np.histogram(img.flatten(), 256, [0,255])
    cdf = hist.cumsum()
    cdf_m = np.ma.masked_equal(cdf, 0)
    cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
    return np.ma.filled(cdf_m, 0).astype('uint8')[img]
def contrast_stretching(img):
    return ((img - img.min()) * 255 / (img.max() - img.min())).astype(np.uint8)

option = input("Nhập I, G, L, H, C: ").upper()
folder = "exercise"
for filename in os.listdir(folder):
    path = os.path.join(folder, filename)
    gray = np.asarray(Image.open(path).convert("L"))
    if option == "I": out = image_inverse(gray)
    elif option == "G": out = gamma_correction(gray)
    elif option == "L": out = log_transform(gray)
    elif option == "H": out = histogram_equalization(gray)
    elif option == "C": out = contrast_stretching(gray)
    else: continue
    plt.imshow(out, cmap='gray'); plt.title(filename); plt.show()


## Câu 2: Menu xử lý tần số (F, L, H)

In [ ]:

def fast_fourier(img):
    return np.abs(fft.fftshift(fft.fft2(img)))

def butterworth_lowpass(img, d0=30, n=2):
    M, N = img.shape
    u0, v0 = M//2, N//2
    H = np.zeros((M, N))
    for u in range(M):
        for v in range(N):
            D = np.sqrt((u - u0)**2 + (v - v0)**2)
            H[u, v] = 1 / (1 + (D/d0)**(2*n))
    F = fft.fftshift(fft.fft2(img))
    G = F * H
    return np.abs(fft.ifft2(fft.ifftshift(G)))

def butterworth_highpass(img, d0=30, n=2):
    M, N = img.shape
    u0, v0 = M//2, N//2
    H = np.zeros((M, N))
    for u in range(M):
        for v in range(N):
            D = np.sqrt((u - u0)**2 + (v - v0)**2)
            H[u, v] = 1 / (1 + (d0/D)**(2*n)) if D != 0 else 0
    F = fft.fftshift(fft.fft2(img))
    G = F * H
    return np.abs(fft.ifft2(fft.ifftshift(G)))

option = input("Nhập F, L, H: ").upper()
folder = "exercise"
for filename in os.listdir(folder):
    path = os.path.join(folder, filename)
    gray = np.asarray(Image.open(path).convert("L"))
    if option == "F": out = fast_fourier(gray)
    elif option == "L": out = butterworth_lowpass(gray)
    elif option == "H": out = butterworth_highpass(gray)
    else: continue
    plt.imshow(out, cmap='gray'); plt.title(filename); plt.show()


## Câu 3: Ngẫu nhiên 1 phép từ Câu 1 áp dụng lên từng kênh RGB

In [ ]:

ops = [image_inverse, gamma_correction, log_transform, histogram_equalization, contrast_stretching]
folder = "exercise"
for filename in os.listdir(folder):
    path = os.path.join(folder, filename)
    rgb = np.asarray(Image.open(path).convert("RGB"))
    out = rgb.copy()
    for ch in range(3):
        func = random.choice(ops)
        out[..., ch] = func(rgb[..., ch])
    plt.imshow(out); plt.title(filename); plt.show()


## Câu 4: Ngẫu nhiên lọc Butterworth + Max/Min Filter

In [ ]:

folder = "exercise"
for filename in os.listdir(folder):
    path = os.path.join(folder, filename)
    rgb = np.asarray(Image.open(path).convert("RGB"))
    out = rgb.copy()
    for ch in range(3):
        g = rgb[..., ch]
        if random.choice(["low", "high"]) == "low":
            out[..., ch] = np.minimum(g, butterworth_lowpass(g)).astype(np.uint8)
        else:
            out[..., ch] = np.maximum(g, butterworth_highpass(g)).astype(np.uint8)
    plt.imshow(out); plt.title(filename); plt.show()
